In [10]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torchaudio
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
cudnn.benchmark = True
plt.ion()   # interactive mode
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
data_transforms = {
    'test': transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3,1,1)),
    ]),
}

data_dir = '../input/pytorchdata/data/librosa' # path 
image_datasets = {'test': torchvision.datasets.ImageFolder(os.path.join(data_dir, 'test'),
                                          data_transforms['test'])}
dataloaders = {'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=1,
                                             shuffle=False, num_workers=2)}
dataset_sizes = {'test': len(image_datasets['test'])}
class_names = image_datasets['test'].classes
print(class_names)

In [13]:
#setup path where model is stored
PATH = '../input/resnet-emotion/emotionRecognition_restnet.pt'

model = torchvision.models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model.load_state_dict(torch.load(PATH, map_location=device))
model.eval()
model = model.to(device)

In [14]:
Y_test = []
predictions = []
for sample, label in dataloaders['test']:
    sample = sample.to(device)
    label = label.cpu().numpy()[0]
    Y_test.append(label)
    #predict with CNN
    outputs = model(sample)
    _, predicted = torch.max(outputs, 1)
    predicted = predicted.cpu().numpy()[0]
    predictions.append(predicted)

In [16]:
# Compute evaluation metrics
UAR = recall_score(Y_test, predictions, average='macro')
WAR = recall_score(Y_test, predictions, average='weighted')
print('UAR =', UAR)
print('WAR =', WAR)
# Confusion matrix
# Angst, Ekel, Freude, Langeweile, Neutral, Traurigkeit, Wut
cm = confusion_matrix(Y_test, predictions, labels=[0,1,2,3,4,5,6,7])
print(cm)

# Compute recall, precision, and F1 score for each class
rec_result = recall_score(Y_test, predictions, average=None, labels=[0,1,2,3,4,5,6,7])
for i, class_ in enumerate(class_names):
    print(f'Recall for {class_} =', rec_result[i]*100, '%')


prec_result = precision_score(Y_test, predictions, average=None, labels=[0,1,2,3,4,5,6,7])
for i, class_ in enumerate(class_names):
    print(f'Precision for {class_} =', prec_result[i]*100, '%')